In [ ]:
#pip install azure.storage.blob

In [18]:
# Retrieve list of blobs from designated container
# Check if blob name is a file name in current directory
# If no same file name in directory, download the file associated with that blobname

import time
from azure.storage.blob import ContainerClient
from azure.storage.blob import BlobServiceClient
import json
from pymongo import MongoClient

blob_service_client_instance = BlobServiceClient(account_url=STORAGEACCOUNTURL, credential=STORAGEACCOUNTKEY)
container = ContainerClient.from_connection_string(conn_str=os.environ.get('AZURE_STORAGE_CONNECTION_STRING'), container_name="container2")

new_files = []
all_files = []

blob_list = container.list_blobs()
for blob in blob_list:
    print(blob.name + '\n')
    all_files.append(blob.name)
    if blob.name not in os.listdir(os.curdir):
        new_files.append(blob.name)
        t1=time.time()
        blob_client_instance = blob_service_client_instance.get_blob_client("container2", blob.name, snapshot=None)
        with open(blob.name, "wb") as my_blob:
            blob_data = blob_client_instance.download_blob()
            blob_data.readinto(my_blob)
        t2=time.time()
        print(("It takes %s seconds to download "+blob.name + '\n') % (t2 - t1))

### Import new data to mongodb, one file at a time

# Making Connection
myclient = MongoClient("mongodb://localhost:27017/")

# database
db = myclient["Sensorlab"]  #update here to switch database

# Create or switch to collection
Collection = db["E4_data"]  #update here to switch collection

for i in range(len(new_files)):
    with open(new_files[i]) as f:
        content = f.readlines()
    file_data = []
    for data in content:
        file_data.append(json.loads(data))

    if isinstance(file_data, list):
        Collection.insert_many(file_data)
    else:
        Collection.insert_one(file_data)
    print(f"Imported {new_files[i]} to the database.")



0_18fc7328361948c0865df2f0c53aa008_1.json

0_2f7120ed65194093a6df6dacd979d706_1.json

0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

It takes 3.7836661338806152 seconds to download 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json

0_4862fd0f886945c6b5fa1ed9b391e807_1.json

0_5e20c79f9efc42fc9c09b4443a6646de_1.json

0_603c31b59afd4c0eb25fade694cdb24c_1.json

0_6bf71ccc6254498bb6e52e10f45490d8_1.json

0_743237a427664f57b35d3bd642572e6d_1.json

0_75d343691ca849bdb190d98b11f7edc9_1.json

0_8223e703445d414c8f34b45d71a9fe35_1.json

0_8d054dcd0a7b487585f77bfee87ff543_1.json

0_a48ce3b7dec3442f963db431ff2e4da5_1.json

0_bcf7c48af48e4ff0bde2b79cf6606216_1.json

0_ddb730a7073646d995d0534ada10ceac_1.json

It takes 3.704059600830078 seconds to download 0_ddb730a7073646d995d0534ada10ceac_1.json

0_f1e1947720fb4072a8b325428be98a10_1.json

Imported 0_36ecfdb7ea4941c2938e3f91a9003ea0_1.json to the database.
Imported 0_ddb730a7073646d995d0534ada10ceac_1.json to the database.


In [15]:
print(("Imported %s to database ")%(all_files[0]))

Imported 0_18fc7328361948c0865df2f0c53aa008_1.json to database 


In [16]:
print(f"Imported {all_files[0]} to database")

Imported 0_18fc7328361948c0865df2f0c53aa008_1.json to database


TypeError: name must be an instance of str

In [1]:
import os
os.environ['AZURE_STORAGE_CONNECTION_STRING'] = "DefaultEndpointsProtocol=https;AccountName=uaholodecksensorlab;AccountKey=GX+Fn1hVo3RDWRGuCxMAVDVFA/maCM2NdGx4Kffv4tWnG6DU8C1NOVH5Rv694e3HVNCmlinkeAKMgnXBvsr7nA==;EndpointSuffix=core.windows.net"

STORAGEACCOUNTURL= "https://uaholodecksensorlab.blob.core.windows.net"
STORAGEACCOUNTKEY= "GX+Fn1hVo3RDWRGuCxMAVDVFA/maCM2NdGx4Kffv4tWnG6DU8C1NOVH5Rv694e3HVNCmlinkeAKMgnXBvsr7nA=="